#Project Imports

In [ ]:
import boto3
import re
from sagemaker import get_execution_role
import sagemaker
import pandas as pd
import numpy as np
import json
import os
import shutil
import tarfile
import tensorflow as tf
from tensorflow import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
import tensorflow.keras as layers
from sklearn.preprocessing import StandardScaler
from sagemaker.tensorflow.model import TensorFlowModel
from sklearn.metrics import r2_score
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils  import predict_signature_def
from tensorflow.python.saved_model import tag_constants
from keras import backend as K
bucket = sagemaker.Session().default_bucket()

role = get_execution_role()

Getting The Data from the local environment

In [ ]:
df=pd.read_csv("mushrooms.csv")

#The data File guaranted to have no null values.

In [ ]:
df.isna().sum()

#Looking at the mushroom Data.csv

In [ ]:
df.head()

#number of unique vlaues at each cloumns

In [ ]:
df.nunique()

We see some data have more than 2 # of unique data - so we need to use get dummies - using HOT encoding

In [ ]:
df=pd.get_dummies(df, columns=['cap-shape','cap-color','cap-surface','odor','gill-color','stalk-root','stalk-surface-above-ring',
            'stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-color','ring-number','ring-type','spore-print-color',
            'population','habitat'])
df.head()

#Here we encoding the Binary features and the class

In [ ]:
le=LabelEncoder()
for item in df.columns:
  df[item]=le.fit_transform(df[item])

df.head()

# we drop the target as y  from dataframe and the rest goes to x as features

In [ ]:
x=df.drop(labels=['class'],axis=1)
y=df[['class']]

# Now we split the data to training and testing, 0.8 and 0.2 ratio respectively. 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=1)

#We are defining the model as Fucntion
where the model is Sequential consist 3 Dense layers.
The data shape (num of records , 112)
activation is 'relu'
and the last layer Sigmoid which make the output bettween 0 - 1 
since this Problem Binary classification.

In [ ]:
def build_model():
    model = Sequential()
    model.add(Dense(32, activation = 'relu', input_dim = x_train.shape[1]))
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(1,  activation = 'sigmoid'))
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [ ]:
model=build_model()

#Now the fun begins with our Fiting method.
we are showing the logs "verbose=1".

In [ ]:
model.fit(x_train,y_train,
          verbose=1,
          epochs=20,
          validation_split=.1
          )

We Got 100% for the accuracy!

#Evaluating the Model using r2_score from the sikit learn Library.
to see if the model overfiting or memorised the data

In [ ]:
r2_score(y_test,model.predict(x_test))

Oh Nice! our Model is Perfect!
with 99% accuracy!

#We are ready to deploy the model to AWS cloud.
we make the directory Named keras_model as the usual protocal. 
and we save the weights to the same path.
then we write our model to Json.

In [ ]:

!mkdir "keras_model"
save_path = "./keras_model/"

model.save_weights(os.path.join(save_path,"model-weights.h5"))

model_json = model.to_json()
with open(os.path.join(save_path,"model.json"), "w") as json_file:
    json_file.write(model_json)

#opening the Json file and loding the model to the loaded_model

In [ ]:
json_file = open('keras_model/'+\
                 'model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json,custom_objects=\
        {"GlorotUniform": tf.keras.initializers.glorot_uniform})

In [ ]:
#loading Weights
loaded_model.load_weights(\
    'keras_model/model-weights.h5')
print("Loaded model from disk")

We make locally directory.
and the expected path for the model.

In [ ]:
# Note: This directory structure will need to be followed - see notes 
# for the next section
model_version = '1'
export_dir = 'export/Servo/' + model_version

deleting anything leaving in that path

In [ ]:

shutil.rmtree(export_dir)

# Build the Protocol Buffer SavedModel at 'export_dir'

In [ ]:
build = builder.SavedModelBuilder(export_dir)

# Creating prediction signature to be used by TensorFlow Serving Predict API

In [ ]:
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, \
    outputs={"score": loaded_model.output})

In [ ]:


with K.get_session() as sess:
    # Save the meta graph and variables
    build.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], \
        signature_def_map={"serving_default": signature})
    build.save()

In [ ]:

with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [ ]:
sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', \
                key_prefix='model')

In [ ]:
#creating train.py
!touch train.py

In [ ]:

sagemaker_model = TensorFlowModel(model_data = 's3://' + \
    sagemaker_session.default_bucket() + '/model/model.tar.gz',
    role = role,
    framework_version = '1.12',
    entry_point = 'train.py')

Deploying the model to the Cloud.

In [ ]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.t2.medium')

In [ ]:
#EndPoint Name
predictor.endpoint

In [ ]:
endpoint_name = "From the previous EndPoint Name"

In [ ]:
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(\
        endpoint_name, sagemaker_session)

In [ ]:
#From inside the AWS 
client = boto3.client('runtime.sagemaker', region_name='us-east-1') 

In [ ]:
#Testing the deployed Predictor with dummiy data 

data = [[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0]]

response = client.invoke_endpoint(EndpointName=endpoint_name, \
                                  Body=json.dumps(data))
response_body = response["Body"]
print(json.loads(response_body.read())['outputs']['score']['floatVal'])

And its ready to Go!